In [16]:
import pandas as pd 
import numpy as np 
import scanpy as sc
import matplotlib.pyplot as plt
import concurrent.futures
import pickle
import warnings
from datetime import date
import hisepy
import os
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed,ProcessPoolExecutor
from tqdm import tqdm
import anndata
import gc
warnings.filterwarnings("ignore")
sc.settings.n_jobs = 60
print("Current working directory:", os.getcwd())

Current working directory: /home/workspace/IHA_Figure_Revision/Figure1/06_DEG_Y1D0_vs_Y2D0


# Read MetaData

In [17]:
meta_data=pd.read_csv("//home/workspace/IHA_Figure_Revision/Dataset/scRNA_meta_data-2024-05-09.csv")

# Assemble Data

In [18]:
meta_data_subset=meta_data[meta_data['sample.visitName'].isin(['Flu Year 1 Day 0','Flu Year 2 Day 0'])]

In [19]:
meta_data_subset=meta_data_subset[meta_data_subset['Covid_exclusion']=='no']

In [20]:
subject=meta_data_subset['subject.subjectGuid'].value_counts()[meta_data_subset['subject.subjectGuid'].value_counts()==2]

In [21]:
meta_data_subset=meta_data_subset[meta_data_subset['subject.subjectGuid'].isin(subject.index.tolist())] 

In [22]:
%%time
file_names= ['/home/workspace/IHA_Figure_Revision/Dataset/scRNA/BRI/h5ad/sample_h5ad/'+x+".h5ad" for x in meta_data_subset['pbmc_sample_id'].tolist()]
adata_list = []
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_file = {executor.submit(sc.read_h5ad, file_name): file_name for file_name in file_names}
    for future in tqdm(as_completed(future_to_file), total=len(file_names)):
        result = future.result()
        if result is not None:
            adata_list.append(result)

100%|██████████| 168/168 [02:44<00:00,  1.02it/s]

CPU times: user 31.3 s, sys: 37.1 s, total: 1min 8s
Wall time: 2min 44s


In [ ]:
adata = anndata.concat(adata_list)
del adata_list 
gc.collect()

# Read Files and filter genes for DESEQ2

In [ ]:
fitlered_gene=pd.DataFrame()
for i in adata.obs['AIFI_L3'].unique():
    print(i)
    adata_subset=adata[adata.obs['AIFI_L3']==i]
    sc.pp.filter_genes(adata_subset, min_cells=round(np.shape(adata_subset.X)[0]*0.1))
    gene_list=pd.DataFrame(list(adata_subset.var.index))
    gene_list.columns=['gene']
    gene_list['AIFI_L3']=i
    fitlered_gene=pd.concat([fitlered_gene,gene_list])

In [ ]:
fitlered_gene.to_csv("filtered_gene.csv")